# Evaluation of a Tree Clustering Algorithm

Given samples $\{\mathbf x_i, y_i \}$, we classify the samples in an *unsupervised* manner using a decision-tree-like algorithm, meaning that the ground truth labels $y_i$ are invisible to the algorithm. We then evaluate the trees by computing the loss-depth curve, which shows at given depth of the tree, how well is the clustering performance.

For example, we can use the impurity as the loss function. Suppose at depth $d$, the tree consists of $m_d$ nodes, each of which contains reigon $\{ R^d_i \}_{i=1}^{m_d}$. Suppose there are $c$ ground truth classes (number of geometries in the scene). The loss the tree at depth $d$ is defined as

$$

L(d) = \sum_{i=1}^{m_d} \mathbb P(\mathbf x \in R^d_i) \cdot \mathrm{Imp}(R^d_i) \\

\mathrm{Imp}(R) = \sum_{j=1}^c - \mathbb P(y = j | \mathbf x \in R) \log \mathbb P(y = j | \mathbf x \in R)

$$

Suppose the count of each class in region $R$ is $\{ n_j \}_{j=1}^c$, then

$$

\mathrm{Imp}(R) = \sum_{j=1}^c - \frac{n_j}{\sum_{k=1}^c n_k} \log \frac{n_j}{\sum_{k=1}^c n_k}

$$

In [3]:
import numpy as np
import matplotlib.pyplot as plt